# Web scraping - Extraccion de la data

Considerar el punto de que:
El web scraping que realizado, se encarga de descargar(link referenciando al archivo parket de la data) los archivos de la pagina web de TCL(Taxis & Limousine Commission), por lo que en realidad podrias decir cumplimos con una premisa del web scraping: la extracción de datos de una fuente externa, pero no realizamos una "disección" o "scraping"(en realidad referie a raspado-raspar) del documento html para luego procesar la data para construir y reorganizar los datos. En este caso, la data ya se obtiene en formato tabular.

In [54]:
import requests
import os
import pandas as pd
import time
from selenium import webdriver # pip install selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# Crear una carpeta para las descargas si no existe
download_dir = os.path.join(os.getcwd(), "descargas")
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
print(download_dir)

#### Web driver de Selenium

In [56]:
# Configura el WebDriver (asegúrate de tener el driver correspondiente)
options = webdriver.ChromeOptions()
options.add_argument("--window-size=800,640")
#Directorio predeterminado para descargar los archivos
prefs = {
    "download.default_directory": download_dir,  # Cambiar a tu directorio de descargas
    "download.prompt_for_download": False,       # Desactivar el cuadro de diálogo de descarga
    "download.directory_upgrade": True,          # Permitir actualizar la carpeta de descarga
    "safebrowsing.enabled": True                 # Activar descarga segura
}
options.add_experimental_option("prefs", prefs)
service = Service(executable_path="chromedriver_win64\chromedriver.exe") #Ruta del chromedriver
options.headless = True  # Enable headless mode
#options.add_argument("--window-size=800,640")
driver = webdriver.Chrome(service=service, options=options)
driver.get('https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page') # URL pagina de la data

# Usar WebDriverWait para esperar hasta que la sección con la clase y el id estén visibles
wait = WebDriverWait(driver, 20)
faq_section = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.faq-answers#faq2023')))

# Buscar todos los enlaces dentro de la sección con class 'faq-answers' y id 'faq2023'
taxi_links = faq_section.find_elements(By.CSS_SELECTOR, 'a[title="Yellow Taxi Trip Records"], a[title="Green Taxi Trip Records"]')

#### Descarga de archivos

In [ ]:
# Descargar cada archivo
for url in taxi_links:
    # Extraer el nombre del archivo de la URL
    url = url.get_attribute('href')
    file_name = url.split("/")[-1]
    file_path = os.path.join(download_dir, file_name)
    # Descargar el archivo
    print(f"Descargando {file_name} ...")
    #response = requests.get(url)
    try:
        response = requests.get(url, timeout=120)  # Aumentar el timeout a 120 segundos
    except requests.exceptions.Timeout:
        print(f"Error: El archivo {file_name} tardó demasiado en responder.")
        continue  # Salta al siguiente archivo si hay un problema de tiempo de espera
    # Verificar si la descarga fue exitosa
    if response.status_code == 200:
        # Guardar el archivo en la carpeta de descargas
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"Archivo descargado y guardado en: {file_path}")
    else:
        print(f"Error al descargar {file_name}: Código de estado {response.status_code}")

In [53]:
# Cierra el driver
driver.quit()